## 1. Импорт библиотек

In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

## 2. Класс для загрузки и аугментации данных

In [ ]:
class AugmentedDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        """
        Инициализация датасета.
        :param img_dir: Путь к директории с изображениями.
        :param transform: Преобразования для аугментации.
        """
        self.img_dir = img_dir
        self.transform = transform
        # Собираем список файлов с нужными расширениями
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith(('.tif', '.jpeg', '.jpg'))]

    def __len__(self):
        """Возвращает количество изображений в датасете."""
        return len(self.img_files)

    def __getitem__(self, idx):
        """
        Загружает изображение и применяет аугментации.
        :param idx: Индекс изображения.
        :return: Аугментированное изображение.
        """
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        image = Image.open(img_path).convert('RGB')  # Загружаем и конвертируем в RGB

        if self.transform:
            image = self.transform(image)  # Применяем аугментации

        return image

## 3. Определение аугментаций

In [ ]:
# Определяем аугментации
augmentation_pipeline = transforms.Compose([
    transforms.Resize((224, 224)),  # Изменение размера до 224x224
    transforms.RandomHorizontalFlip(p=0.5),  # Горизонтальное отражение с вероятностью 50%
    transforms.RandomVerticalFlip(p=0.5),  # Вертикальное отражение с вероятностью 50%
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Коррекция яркости и контрастности
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),  # Случайная обрезка
    transforms.RandomRotation(30),  # Поворот на угол до 30 градусов
    transforms.ToTensor(),  # Преобразование в тензор
])

## 4. Создание датасета и DataLoader

In [ ]:
# Путь к директории с изображениями
img_dir = 'path_to_your_images'

# Создаем датасет с аугментациями
dataset = AugmentedDataset(img_dir, transform=augmentation_pipeline)

# Создаем DataLoader для пакетной обработки
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

## 5. Визуализация аугментированных изображений

In [ ]:
def show_images(images, nrow=4):
    """
    Визуализация изображений.
    :param images: Тензор с изображениями.
    :param nrow: Количество изображений в строке.
    """
    images = images.numpy().transpose((0, 2, 3, 1))  # Преобразуем тензор в формат для matplotlib
    plt.figure(figsize=(10, 10))
    for i in range(len(images)):
        plt.subplot(len(images) // nrow + 1, nrow, i + 1)
        plt.imshow(images[i])
        plt.axis('off')
    plt.show()

# Пример визуализации
images = next(iter(dataloader))
show_images(images)

## Комментарии
Избежание избыточной аугментации: Мы ограничили поворот до 30 градусов и добавили случайное отражение с вероятностью 50%.
Минимальный шум: Мы не добавили шум, так как torchvision.transforms.RandomNoise отсутствует в стандартной библиотеке. 
Если нужно, можно реализовать кастомное добавление шума.
Поддержка форматов: Мы предусмотрели загрузку изображений в форматах .tif, .jpeg, .jpg.

## 7. Интеграция в обучение

In [ ]:
# Пример использования в цикле обучения
for epoch in range(10):  # 10 эпох
    for batch in dataloader:
        # batch - это тензор с аугментированными изображениями
        # Здесь можно добавить код для обучения модели
        pass